## Dependencies

In [1]:
import pandas as pd
import numpy as np
import os
import glob

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15, 5)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [10]:
indataPath = os.path.join(os.pardir, "indata")

ddfRootPath = os.path.join(indataPath, "ddf--sodertornsmodellen")
ddfSrcPath = os.path.join(ddfRootPath, "ddf--sodertornsmodellen--src")

superPath = os.path.join(indataPath, "supermappen")

ddfOutputPath = os.path.join(os.pardir, 'ddf--sodertornsmodellen-new', 'ddf--sodertornsmodellen--src')

## Helpers

In [3]:
from helpers import (readMaster, appendNewDatapoints, byGender, baskod2010tobasomrade, plotcombined)

## Läs in MASTER excel filen

In [4]:
master = readMaster()

----

----

----

## Lookups

### Baskod2010 => Baskod2000

In [5]:
baskodkey = pd.read_excel(os.path.join(ddfRootPath, 'etl', 'source', '161115 A7 utan formler.xlsx'), skiprows=[0,1,2,3,4,5,6], converters={2010: lambda x: str(x)})
baskodkey = baskodkey[[2010, 'BASKOD2000']]
baskodkey = baskodkey.rename(columns={2010: 'BASKOD2010'})
baskodkey['BASKOD2010'] = baskodkey['BASKOD2010'].astype(str).astype(int)

In [6]:
baskodkey.head(5)

,BASKOD2010,BASKOD2000
0,1010110,1010110
1,1010120,1010120
2,1010130,1010130
3,1010140,1010140
4,1010201,1010201


In [7]:
print(baskodkey.shape[0])
print(baskodkey['BASKOD2010'].nunique())
print(baskodkey['BASKOD2000'].nunique())

1364
1364
1299


### Baskod2000 => Basomr entity name 

In [13]:
entityKey = pd.read_csv(os.path.join(ddfOutputPath, 'ddf--entities--basomrade.csv'))
entityKey = entityKey.rename(columns={'baskod2000': 'BASKOD2000'})
entityKey['basomrade'] = entityKey['basomrade'].astype(str)

entityKey
# entityKey = entityKey[['basomrade', 'BASKOD2000']]
# entityKey

,basomrade,name,municipality,is--basomrade,size,baskod2010
0,4111110_norra_runby,Norra Runby,0114_upplands_vasby,True,big,4111110
1,4111120_sodra_runby,Södra Runby,0114_upplands_vasby,True,big,4111120
2,4111130_runby_lovsta,Runby Lövsta,0114_upplands_vasby,True,big,4111130
3,4111140_prastgardsmarken,Prästgårdsmarken,0114_upplands_vasby,True,big,4111140
4,4111900_eds_glesbygd,Eds glesbygd,0114_upplands_vasby,True,big,4111900
5,4122100_lowenstromska_sjukhuset,Löwenströmska Sjukhuset,0114_upplands_vasby,True,big,4122100
6,4122210_norra_vilunda,Norra Vilunda,0114_upplands_vasby,True,big,4122210
7,4122230_apoteksskogen,Apoteksskogen,0114_upplands_vasby,True,big,4122230
8,4122240_vasbyskogen,Väsbyskogen,0114_upplands_vasby,True,big,4122240
9,4122310_korpkulla,Korpkulla,0114_upplands_vasby,True,big,4122310


In [ ]:
entityKey.nunique()

----

----

----

# Lista kolumner i masterfilen

In [ ]:
mas2014 = master[master['year'] >= pd.to_datetime('2014')]
mas2014 = mas2014.rename(columns = {
    'BASKODER': 'BASKOD2010'
})

In [ ]:
masterCols = mas2014.columns.values[5:-2]#.tolist()
len(masterCols)
masterCols

In [ ]:
for col in  masterCols:
    coldf = mas2014[['BASKOD2010', 'year', col]]
    coldf = pd.merge(coldf, baskodkey, on='BASKOD2010', how='left')
    coldf = pd.merge(coldf, entityKey[['BASKOD2000', 'basomrade']], on='BASKOD2000', how='left')
    coldf = coldf.dropna(how='any')
#     print(coldf)
#     print(master[['BASKODER', col]])

## Kombinera `.csv` med MASTER 

In [ ]:
csv = pd.read_csv('../indata/supermappen/Tabell2A_FB7bas.csv', encoding='latin1' , sep=';', decimal=',')
csv['year'] = pd.to_datetime('2017')
csv = csv.rename(columns={
    "Basomrade": "BASKOD2010",
    "Hyresrätt": "antal_hyres",
    "Bostadsrätt": "antal_bostadsr",
    "Äganderätt": "antal_aegender",
    "Uppgift_saknas": "Lght_Uppgift saknas",
    "TOTALT": "Lght total"
})

subsetmas = master[['BASKODER','year', 'Lght_Hyresrätt', 'Lght_Bostadsrätt','Lght_Äganderätt','Lght_Uppgift saknas','Lght total']]
subsetmas = subsetmas.rename(columns = {
    "Lght_Hyresrätt": "antal_hyres",
    "Lght_Bostadsrätt": "antal_bostadsr",
    "Lght_Äganderätt": "antal_aegender"
})

In [ ]:
mas2012 = subsetmas[subsetmas['year'] >= pd.to_datetime('2012')]
mas2012 = mas2012.rename(columns = {
    'BASKODER': 'BASKOD2010'
})

In [ ]:
concat = pd.concat([csv, mas2012], sort=False)

## Summera efter år och Basområde2000 kod

In [9]:
# join the dataframe with the baskod2010 => baskod2000 key dataframe
# merged = pd.merge(concat, baskodkey, on='BASKOD2010', how='left')

In [10]:
# merged = pd.merge(merged, entityKey[['BASKOD2000', 'basomrade']], on='BASKOD2000', how='left')

In [11]:
merged = baskod2010tobasomrade(concat)

NameError: name 'concat' is not defined

In [12]:
# sidospår: de här basområdena saknas helt i entitiesfilen
notinc = merged[merged['basomrade'].isnull()].groupby('BASKOD2000').sum().reset_index()[['BASKOD2000', 'BASKOD2010']]
notinc['notinc'] = True
notinc.to_csv('experiment/notincluded.csv', index=False)

NameError: name 'merged' is not defined

In [20]:
# sum values by year and baskod2000
subset = merged[['year','BASKOD2000', 'basomrade', 'antal_hyres', 'antal_bostadsr', 'antal_aegender', 'Lght total']]
summed = pd.DataFrame(subset.groupby(['year','basomrade']).sum()).reset_index()

In [21]:
summed['year'] = summed['year'].apply(lambda t: t.year)

In [22]:
# ta bort rader som saknar data
summed = summed.dropna(how='any')

In [23]:
summed[['year', 'Lght total']].groupby('year').sum()

,Lght total
year,
2012,943331.0
2013,950892.0
2014,959773.0
2015,971836.0
2016,990371.0
2017,1028961.0


## Räkna andelar och exportera

#### Andel hyresrätter

In [24]:
summed['andel_hyres'] = summed['antal_hyres'] / summed['Lght total']
summed['andel_hyres'] = summed['andel_hyres'].round(4)
summed['andel_hyres'] = summed['andel_hyres'].astype(str).replace('0.0', '0')
summed['andel_hyres'] = summed['andel_hyres'].astype(str).replace('1.0', '1')
summed['andel_hyres'] = summed['andel_hyres'].astype(str).replace('nan', '')

In [25]:
summed[['basomrade', 'year', 'andel_hyres']].to_csv(os.path.join(ddfOutputPath, 'ddf--datapoints--andel_hyres--by--basomrade--year.csv'), index=False)

#### Andel bostadsrätter

In [26]:
summed['andel_bostadsr'] = summed['antal_bostadsr'] / summed['Lght total']
summed['andel_bostadsr'] = summed['andel_bostadsr'].round(4)
summed['andel_bostadsr'] = summed['andel_bostadsr'].astype(str).replace('0.0', '0')
summed['andel_bostadsr'] = summed['andel_bostadsr'].astype(str).replace('1.0', '1')
summed['andel_bostadsr'] = summed['andel_bostadsr'].astype(str).replace('nan', '')

In [27]:
summed[['basomrade', 'year', 'andel_bostadsr']].to_csv(os.path.join(ddfOutputPath, 'ddf--datapoints--andel_bostadsr--by--basomrade--year.csv'), index=False)

#### Andel äganderätter

In [28]:
summed['andel_aegender'] = summed['antal_aegender'] / summed['Lght total']
summed['andel_aegender'] = summed['andel_aegender'].round(4)
summed['andel_aegender'] = summed['andel_aegender'].astype(str).replace('0.0', '0')
summed['andel_aegender'] = summed['andel_aegender'].astype(str).replace('1.0', '1')
summed['andel_aegender'] = summed['andel_aegender'].astype(str).replace('nan', '')

In [29]:
summed[['basomrade', 'year', 'andel_aegender']].to_csv(os.path.join(ddfOutputPath, 'ddf--datapoints--andel_aegender--by--basomrade--year.csv'), index=False)